In [129]:
import pdfplumber
import regex as re
import pyawk  # so far not used
import spacy
import glob

nlp = spacy.load("de_core_news_sm")
from typing import List, Tuple

In [617]:
pdfs = glob.glob("../data/*.pdf")
# print(pdfs)
pdf_file = open(pdfs[4], "rb")
### imported pyawk but haven't set it up yet

### Goal:
# input: raw PDF of antwurf
# output: list of (law title, change request text) pairs

## TODO:
# Spacy is messing up some of the scentences (cutting them too short)
# consider going back to manually taking first two lines unles second line start with "In" or "§"

# next wishful steps:
# Analyse entwurf patterns, common, less common etc.
# API to grab law text using that damn title
# API to grab entwurfs

In [618]:
def read_pdf_law(filename: str) -> str:
    """Get the raw text from the pdfs."""
    # read all pages from provided pdf
    pdf_file_obj = pdfplumber.open(filename)

    # join the pages
    return "".join(
        [page for page in [page.extract_text() for page in pdf_file_obj.pages] if page]
    )


change_law_raw = read_pdf_law(pdf_file)

In [619]:
def extract_raw_proposal(text: str) -> str:
    text = re.split("\nArtikel 1.*?\n", text, maxsplit=1)[1].split("Begründung")[0]
    return text


change_law_extract = extract_raw_proposal(change_law_raw)

In [620]:
def extract_seperate_change_proposals(text: str) -> List[str]:
    proposals = []
    proposals.extend(re.split("Artikel\s{1,2}([1-9]|[1-9][0-9]|100)\s{0,2}\n", text))

    artikels = re.findall("Artikel\s{1,2}([1-9]|[1-9][0-9]|100)\s{0,2}\n", text)
    artikels_int = [int(artikel) for artikel in artikels]
    # compute the sum of the artikel numbers using the formula for an arithmetic series
    # using only the first and last artikel numbers, and the length of the list
    math_sum_artikels = len(artikels_int) * (artikels_int[0] + artikels_int[-1]) / 2
    # compute the sum of the artikel numbers directly
    raw_sum_artikels = sum(artikels_int)

    # uncomment next line to see if see if the next condition is met or not
    ## print(int(raw_sum_artikels), int(math_sum_artikels))

    # if the direct sum is larger than the arithmetic series, then we had an artikel from some paragraph
    # sneak into the titel list, by mentioned as "Artikel [1-100]" just before a line break :(((
    # in this case we need to go through the list and check that the artikel numbers are regularly increasing by 1
    if int(raw_sum_artikels) != int(math_sum_artikels):
        proposal_list = []
        proposal_list.append(proposals[0])
        artikel_number = 2
        for prop_index, proposal in enumerate(proposals):
            try:
                if int(proposal) == artikel_number:
                    artikel_number += artikel_number
                    proposal_list.append(proposals[prop_index + 1])
            except:
                pass
    else:
        proposal_list = proposals[
            0::2
        ]  # remove odd indexed items since they contain the artikel number only and we don't need it

    return proposal_list


proposals_list = extract_seperate_change_proposals(change_law_extract)

In [621]:
def extract_law_titles(props: List[str]) -> List[str]:
    raw_titles_list = [
        list(nlp(proposal.replace("\n", " ")).sents)[0].text
        for proposal in proposals_list
    ]
    titles_clean_par_sign = [re.split("§", title)[0] for title in raw_titles_list]
    titles_clean_aenderung = []
    for title in titles_clean_par_sign:
        if re.search(r"Änderung \b(de[rs])\b ", title) is not None:
            titles_clean_aenderung.append(
                re.split(r"Änderung \b(de[rs])\b ", title)[2].strip()
            )
        else:
            titles_clean_aenderung.append(title.strip())
    return titles_clean_aenderung


law_titles = extract_law_titles(proposals_list)

In [622]:
def remove_inkrafttreten(titles: List[str], props: List[str]) -> Tuple[List, List]:
    if re.search("^Inkrafttreten", titles[-1]) is not None:
        titles = titles[:-1]
        props = props[:-1]

    return titles, props


law_titles, proposals_list = remove_inkrafttreten(law_titles, proposals_list)

In [623]:
pdf_file, law_titles, proposals_list

(<_io.BufferedReader name='../data/1928399.pdf'>,
 ['Zivilprozessordnung',
  'Strafprozessordnung',
  'Gesetzes  über  das Verfahren  in Familiensachen und in  Angelegenheiten der   freiwilligen Gerichtsbarkeit',
  'Elektronischer-Rechtsverkehr-Verordnung',
  'Arbeitsgerichtsgesetzes',
  'Arbeitsgerichtsgesetzes',
  'Arbeitsgerichtsgesetzes zum  1. Januar 2022',
  'Arbeitsgerichtsgesetzes zum  1. Januar 2026',
  'Sozialgerichtsgesetzes',
  'Sozialgerichtsgesetzes zum  1. Januar 2022',
  'Sozialgerichtsgesetzes zum  1. Januar 2026',
  'Verwaltungsgerichtsordnung',
  'Verwaltungsgerichtsordnung zum  1. Januar 2022',
  'Verwaltungsgerichtsordnung zum  1. Januar 2026',
  'Finanzgerichtsordnung',
  'Finanzgerichtsordnung zum  1. Januar 2022',
  'Finanzgerichtsordnung zum  1. Januar 2026',
  'Bundesrechtsanwaltsordnung',
  'Beurkundungsgesetzes',
  'Gesetzes  über die  Tätigkeit  europäischer Rechtsanwälte in Deutschland',
  'Verordnung zur Einführung von Vordrucken für das Mahnverfahren',
 

In [ ]:
### following code is old and will get cleaned up soon

In [121]:
with pdfplumber.open(pdf_file) as pdf:
    text_of_change = [page.extract_text() for page in pdf.pages]

In [21]:
# quirky mehtod to find the first page where the Entwurf starts in the PDF
# it picks the first page that has the a stand alone title "Artikel 1" on it's own line,
# no matter how many spaces it has before the line break at the end
for pagenum, page in enumerate(text_of_change):
    if len(re.split("\nArtikel 1.+\n", page, maxsplit=1)) == 2:
        first_page = pagenum
        break

last_page = len(text_of_change)

# quirky mehtod to find the last page where the Entwurf ends in the PDF
# it picks the last page before a page that has the word "Begründung" in it
for page in range(first_page, last_page):
    if len(text_of_change[page].split("Begründung")) > 1:
        last_page = page - 1
        break

# uncomment next line to manually check if the fist and last page were identified correctly
print(first_page, last_page)

4 5


In [5]:
# goes over the entwurf pages, and makes a list of all the 2 lines of text after the patten "Artikel [number] \n",
# also in case there is more than one change in a page

title_change_of_this_law = []
for page in range(first_page, last_page + 1):
    has_artikel = len(
        re.split("Artikel [0-9]+ +\n", text_of_change[page])
    )  # the len() of the split is 1 is there are no artikles, 2, if there is 1, and
    # increases by 2 (odds) for every additional artikel
    # print(page,has_artikel) # used for debugging only, can be removed
    if has_artikel >= 2:
        for artikel in range(1, has_artikel, 2):
            title_change_of_this_law.extend(
                re.search(
                    ".+\n.+\n",
                    re.split("Artikel [0-9]+ +\n", text_of_change[page])[artikel],
                ).captures()
            )

# the logic for this funciton is as follows:
# in each page, each time the pattern "Artikel [number] \n" appears,
# note that number regexed any number of digits, this could be limited in the future to only 1-2 or 3?! which is more reasonable,
# we split the string and take the part that is after the split (if there is only one split that's 1, if there is more we loop on the odd numbers to
# always take the part after the split), we then use re.search to take the first two scentences using a pattern that take any number of charcters
# before a new line (twice). Since re.search() returns an object we use .capture to to grab the match of that pattern from the returned object.
# since that that match is a list with one string in it, we use .extend() so that our final input is one flat list with all the strings we recovered.
# huzzah.

In [6]:
title_change_of_this_law

['Änderung des Bürgerlichen Gesetzbuchs\nDas Bürgerliche Gesetzbuch in der Fassung der Bekanntmachung vom 2. Januar 2002 (BGBl. \n',
 'Änderung des Einführungsgesetzes zum Bürgerlichen Gesetzbuche\nDem Artikel 229 des Einführungsgesetzes zum Bürgerlichen Gesetzbuche in der Fassung der \n']

In [7]:
# this is an unfinished code chunk
# the following 2 lines only remove the last line break from teh end of the 2nd scentence
[change_title.rstrip() for change_title in title_change_of_this_law]

# the goal is to add a simple logic that capture a second line when it is relevant, e.g. if it start with "und " or if it is no longer than 3 words.
# this won't catch all cases, but it's a start
# more logic to consider: don't take a 2nd line that starts with "§" or "In"

# strip the begining from "Änderung des" or "Weitere Änderung des"

['Änderung des Bürgerlichen Gesetzbuchs\nDas Bürgerliche Gesetzbuch in der Fassung der Bekanntmachung vom 2. Januar 2002 (BGBl.',
 'Änderung des Einführungsgesetzes zum Bürgerlichen Gesetzbuche\nDem Artikel 229 des Einführungsgesetzes zum Bürgerlichen Gesetzbuche in der Fassung der']

In [8]:
# re.search('.+\n.+\n',re.split('Artikel [0-9]+  \n',text_of_change[6])[1]).captures()
# re.split('Artikel [0-9]+ \n',text_of_change[6])[1]
re.split("Artikel [0-9]+  \n", text_of_change[6])

['- 3 - Drucksache 683/21\nBegründung: \nA.  Allgemeiner Teil \nI.  Anlass und Ziel des Gesetzesentwurfs\nZiel des Gesetzesentwurfes ist es, die Position von Mieterinnen und Mietern zu stär-\nken, indem der Möblierungszuschlag in Gebieten mit einem angespannten Wohnungs-\nmarkt transparent geregelt wird, wodurch eine Umgehung der Mietpreisbremse verhin-\ndert werden soll. Zusätzlich wird die Position für Mieterinnen und Mieter bei der Ver-\nmietung von Wohnraum zum vorübergehenden Gebrauch in Gebieten mit einem an-\ngespannten Wohnungsmarkt dadurch gestärkt, dass bei Vermietungen über einen Zeit-\nraum von sechs Monaten oder mehr nunmehr regelmäßig die in § 549 Abs. 2 BGB \ngenannten Mieterschutzvorschriften gelten. Sofern die Vermieterinnen und Vermieter \nsich auf den Ausschlusstatbestand des § 549 Abs. 2 Nr. 1 BGB berufen möchten, \nmüssten diese die Regelvermutung durch Erbringung eines Nachweises entkräften, \nder trotz dieses langen Zeitraums ausnahmsweise die Annahme einer Vermi

In [269]:
title_change_of_this_law = []
for page in range(first_page, last_page + 1):
    has_artikel = len(text_of_change[page].split("Artikel "))
    if has_artikel >= 2:
        for artikel in range(
            1, has_artikel - 1, 2
        ):  # consider using re.split('Artikel [0-9]+ \n') instead
            title_change_of_this_law.append(
                text_of_change[page].split("Artikel ")[artikel].split(" \n")[1:3]
            )
title_change_of_this_law

[['Änderung der Zivilprozessordnung',
  'Die Zivilprozessordnung in der Fassung der Bekanntmachung vom 5. Dezember 2005'],
 ['Änderung des Gesetzes über das Verfahren in Familiensachen',
  'und in Angelegenheiten der freiwilligen Gerichtsbarkeit'],
 ['Änderung der Elektronischer-Rechtsverkehr-Verordnung',
  'Die  Elektronischer-Rechtsverkehr-Verordnung  vom  24.  November  2017  (BGBl. I'],
 ['Änderung des Arbeitsgerichtsgesetzes',
  'Das Arbeitsgerichtsgesetz in der Fassung der Bekanntmachung vom 2. Juli 1979'],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes',
  'Das Arbeitsgerichtsgesetz, das zuletzt durch '],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes zum 1. Januar', '2022'],
 ['Weitere Änderung des Arbeitsgerichtsgesetzes zum 1. Januar', '2026'],
 ['Änderung des Sozialgerichtsgesetzes',
  'Das Sozialgerichtsgesetz in der Fassung der Bekanntmachung vom 23. September'],
 ['Weitere Änderung des Sozialgerichtsgesetzes zum 1. Januar 2022',
  'In § 65d Satz 2 des Sozialgerichtsgese

In [17]:
with pdfplumber.open(pdfFileObj) as pdf:
    for pageNum in pdf.pages()
	name_of_law_to_change = pdf.pages[6].extract_text().split('Artikel 1')[1].split('\n')[1]
    print(name_of_law_to_change)

Änderung der Zivilprozessordnung 


In [179]:
text_of_change[24].split("Artikel ")[1].split("\n")[1]

'Inkrafttreten '

In [167]:
len(text_of_change[6].split("Artikel "))

3

In [188]:
len(title_change_of_this_law[3])

2

In [73]:
text_of_change[4]

'Bundesrat  Drucksache   145/21  \n  \n  \n \n  12.02.21 \n  R - AIS - Fz - In \n \nB\nGesetzentwurf  R\nFu\nder Bundesregierung \nss \nEntwurf  eines  Gesetzes  zum  Ausbau  des  elektronischen \nRechtsverkehrs  mit  den  Gerichten  und  zur  Änderung  weiterer \nprozessrechtlicher Vorschriften \nBundesrepublik Deutschland             Berlin, 12. Februar 2021 \n      Die Bundeskanzlerin \n \nAn den \nPräsidenten des Bundesrates \nHerrn Ministerpräsidenten \nDr. Reiner Haseloff \n \nSehr geehrter Herr Präsident, \n \nhiermit übersende ich gemäß Artikel 76 Absatz 2 des Grundgesetzes den von der \nBundesregierung beschlossenen \n \nEntwurf eines Gesetzes zum Ausbau des elektronischen \nRechtsverkehrs mit den Gerichten und zur Änderung \nweiterer prozessrechtlicher Vorschriften  \n \nmit Begründung und Vorblatt. \n \nFederführend ist das Bundesministerium der Justiz und für Verbraucherschutz. \n \nDie  Stellungnahme  des  Nationalen  Normenkontrollrates  gemäß  § 6  Absatz 1 \nNKRG ist al

In [89]:
text_of_change[5] is None

True